In [1]:
from datetime import datetime
import numpy as np
import tensorflow as tf
from tensorflow import keras

# from IPython.core.interactiveshell import InteractiveShell
# import matplotlib.pyplot as plt
# import seaborn as sns

In [2]:
# 配置项
# # 这个要放到设置中文之前否则还是小方框
# plt.style.use("seaborn")

# # 指定默认字体 用来正常显示中文标签
# plt.rcParams['font.sans-serif'] = ['SimHei']
# # 解决保存图像是负号'-'显示为方块的问题
# plt.rcParams['axes.unicode_minus'] = False

# #全部行都能输出
# InteractiveShell.ast_node_interactivity = "all"

In [3]:
def preprocess_data(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255
    y = tf.cast(y, dtype=tf.int32)
    return x, y

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [5]:
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.map(preprocess_data).shuffle(10000).batch(128).repeat(10)

db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.map(preprocess_data).shuffle(10000).batch(128)

In [6]:
model = keras.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dropout(rate=0.3), # 30%的连接节点弃用
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10)
])

model.build(input_shape=[None, 28 * 28])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  401920    
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  131328    
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  32896     
_________________________________________________________________
dropout_2 (Dropout)          multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  8

In [7]:
# 创建输出日志 用于可视化
current_time = datetime.now().strftime('%Y%m%d-%H%M%S')
log_dir = 'logs/' + current_time
summary_writer = tf.summary.create_file_writer(logdir=log_dir)

In [8]:
# 优化器
optimizer = keras.optimizers.Adam(0.01)

# 统计准确率
accuracy_metrics = keras.metrics.Accuracy()

# 开始训练
for step, (x, y) in enumerate(db_train):
    x = tf.reshape(x, [-1, 28 * 28])
    with tf.GradientTape() as tap:
        y = tf.cast(y, dtype=tf.int32)
        y = tf.one_hot(y, depth=10)
        y_pred = model(x)
        loss = keras.losses.categorical_crossentropy(y, y_pred, from_logits=True)
        loss = tf.reduce_mean(loss)
        
        # 正则化
        loss_regularization = []
        for p in model.trainable_variables:
            loss_regularization.append(tf.nn.l2_loss(p))
         
        # l2正则化 左右参数相加求和
        loss_regularization = tf.reduce_sum(tf.stack(loss_regularization))
        
        loss = loss + 0.001 * loss_regularization
    
    # 对参数球梯度 更新参数
    grads = tap.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    if step % 100 == 0:
        print(step, 'loss:', float(loss), 'loss_regularization:', float(loss_regularization)) 
    
    if step % 500 == 0:
        accuracy_metrics.reset_states()
        
        for (x_, y_) in db_test:
            x_ = tf.reshape(x_, [-1, 28*28])
            y_pred = model(x_)
            y_pred = tf.argmax(y_pred, axis=1, output_type=tf.int32)
            
            accuracy_metrics.update_state(y_, y_pred)
            
        print('【正确率 accuracy_metrics = %f】' % (accuracy_metrics.result().numpy()))    
 
        with summary_writer.as_default():
            tf.summary.scalar('test_accuracy', accuracy_metrics.result().numpy(), step=step)

            
            
        
        


0 loss: 2.947890520095825 loss_regularization: 637.0550537109375
【正确率 accuracy_metrics = 0.097400】
100 loss: 0.5937192440032959 loss_regularization: 324.5043640136719
200 loss: 0.4343646466732025 loss_regularization: 309.3497619628906
300 loss: 0.48443251848220825 loss_regularization: 284.9587097167969
400 loss: 0.45334550738334656 loss_regularization: 279.78680419921875
500 loss: 0.45048758387565613 loss_regularization: 261.0043640136719
【正确率 accuracy_metrics = 0.927800】
600 loss: 0.39158862829208374 loss_regularization: 256.7874755859375
700 loss: 0.4429070055484772 loss_regularization: 265.77374267578125
800 loss: 0.4422701597213745 loss_regularization: 240.0529022216797
900 loss: 0.5517773628234863 loss_regularization: 252.266845703125
1000 loss: 0.4606044590473175 loss_regularization: 235.90267944335938
【正确率 accuracy_metrics = 0.947500】
1100 loss: 0.42014020681381226 loss_regularization: 236.43417358398438
1200 loss: 0.4014856219291687 loss_regularization: 252.77291870117188
1300 